Libraries

In [2]:
import os
from mistralai import Mistral
from dotenv import load_dotenv
import pandas as pd
load_dotenv()
import time
from pymongo import MongoClient
import json

Env and client setup

In [3]:

api_key = os.environ["mistral_key"]
agent_id = os.environ["agent_id"]
db_url = os.environ["DB_URL"]
model = "open-mistral-nemo"
client = Mistral(api_key=api_key)
mongo_client = MongoClient(db_url)
db  = mongo_client.reddit
post_db = db['post_new']

Load Data from MonogoDB

In [4]:
rec = post_db.find()
rec = list(rec)
print(rec[0]) 

{'_id': ObjectId('67558db3eeb7770cf0a37c63'), 'id': '1h9gtbi', 'title': 'Do you think this startup is worth continuing?', 'selftext': "1,006 people came to my app and I made 9 sales. My product is $9.99 and it's a one-time payment. My profit rate is 90%. Do you think this is a good rate or is it not worth continuing? I haven't done any advertising so far, I just made 2 reddit posts without naming the app and in the comments they asked for the name of the app and I posted the link. Do you think I should advertise to influencers? I need help.", 'ups': 1, 'subreddit': 'startups', 'created_utc': 1733656859.0, 'num_comments': 6, 'url': 'https://www.reddit.com/r/startups/comments/1h9gtbi/do_you_think_this_startup_is_worth_continuing/', 'response': 'question'}


Local Data import

In [ ]:
df = pd.read_csv('startups_hot.csv')
df2 = df[['title','selftext']]
df3 = df2[['title','selftext']][:20]
col = list(df.columns)

Update Records in mongodb

In [5]:
def update_post(post):
    try:
        post_db.update_one({'id':post['id']},{'$set':post},upsert=True)
        print(f'Post updated [{post["id"]}]')
    except Exception as e:
        print(e)

Basic Classification with Mistral

In [ ]:
count = 0
for i in rec:
    chat_response = client.agents.complete(
    agent_id=agent_id,
    messages=[
        {
            "role": "user",
            "content": f"title: {i['title']},body: {i['selftext']}",
        },
    ],
)
    response = eval(chat_response.choices[0].message.content)
    count += 1
    print(f'{count}. title:{i['title']},response:{response['message']}')
    i['response'] = response['message']
    update_post(i)
    time.sleep(5)    

In [6]:
def response_gen(post,agent_id):
    chat_response = client.agents.complete(
    agent_id=agent_id,
    messages=[
        {
            "role": "user",
            "content": f"title: {post['title']},body: {post['selftext']}",
        },
    ],
)
    response = eval(chat_response.choices[0].message.content)
    post['response'] = response['message']
    update_post(post)    